In [11]:
from my_tokenizers2 import SmilesTokenizer
import numpy as np
import torch
tokenizer = SmilesTokenizer(vocab_file ='mol2vec_vocabs.txt')


Setting 'max_len_single_sentence' is now deprecated. This value is automatically set up.
Setting 'max_len_sentences_pair' is now deprecated. This value is automatically set up.


In [12]:
from transformers import ElectraModel, ElectraConfig,ElectraForPreTraining,ElectraForMaskedLM
emb_dim = 256
output_dir="./fingerprints_smile_output256/full"
electra_model = ElectraForMaskedLM.from_pretrained(output_dir)

In [13]:
import numpy as np
import csv 
trfile = open('../original-dataset/regression/FreeSolv-remove-other-columes.csv', 'r')
reader = csv.reader(trfile)
next(reader)
smiles,dataY,all_smiles = [], [], []
for raw in reader:
    # print(raw)
    dataY.append(float(raw[-2]))
    smiles = str(raw[-1])
    all_smiles.append(smiles)


    

In [14]:
#before removing duplications
print(len(all_smiles))
print(len(dataY))

642
642


In [15]:
from collections import Counter, defaultdict
def duplicates(lst):
    cnt= Counter(lst)
    return [key for key in cnt.keys() if cnt[key]> 1]
dups = duplicates(all_smiles)


In [16]:
print(len(dups))

0


In [17]:
newY = []
newSm = []
for i,x in enumerate(all_smiles):
    newY.append(dataY[i])
    newSm.append(x)

In [18]:
#after removing duplications
print(len(newSm))
print(len(newY))

642
642


In [19]:
from tqdm import tqdm
from rdkit import Chem
from mol2vec.features import mol2alt_sentence

#smiles_seqs_emb = {}
#smiles_aug_dict = {}

embX = []
MAX_SMI_LEN = 256

for item in tqdm(newSm):
    
    smiles = item
    aas = Chem.MolFromSmiles(item) 
    sentence = mol2alt_sentence(aas, 1)
    sentence = ' '.join(map(str, sentence)) 
    input_ids = tokenizer(sentence, return_tensors="pt")["input_ids"]
    if len(input_ids[0]) > MAX_SMI_LEN:
        print(smiles)
        input_ids = input_ids[:, :MAX_SMI_LEN]
        input_ids[:,255:256] = 3

    outputs = electra_model(input_ids, labels=input_ids)
    embX.append(outputs[2][0].detach().numpy()[0][1:-1])

        
        

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 642/642 [00:12<00:00, 50.47it/s]


In [20]:
np.save('../encoded-dataset/regression/FreeSolv_embX_fullBERT', embX)
np.save('../encoded-dataset/regression/FreeSolv_Y_fullBERT', newY)